# Task 1

In [ ]:
import re
import numpy as np

def list2Arr(lis, wordPos):
    wcnt = np.zeros(len(wordPos))
    for word in lis:
        try:
            wcnt[wordPos[word]] += 1
        except:
            continue
    return wcnt

def tf_idf(url):
    # Data Loading
    corpus = sc.textFile(url) ## url
    # each entry in validLines will be a line from the text file
    validLines = corpus.filter(lambda x: 'id' in x)
    # now we transform it into a bunch of (docID, text) pairs
    keyAndText = validLines.map(lambda x :(x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:]))
    # now we split the text in each (docID, text) pair into a list of words
    # after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
    # we have a bit of fancy regular expression stuff here to make sure that we do not
    # die on some of the documents
    regex = re.compile('[^a-zA-Z]')
    keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))
    # now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
    # to ("word1", 1) ("word2", 1)...
    allWords = keyAndListOfWords.flatMap(lambda x:((j, 1) for j in x[1]))
    # now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
    allCounts = allWords.reduceByKey(lambda a, b: a + b)
    # and get the top 20,000 words in a local array
    # each entry is a ("word1", count) pair
    topWords = allCounts.top(20000, lambda x : x[1])
    Dict_wPos = {}
    for i in range(len(topWords)):
        Dict_wPos[topWords[i][0]] = i
    T1res = keyAndListOfWords.map(lambda x:(x[0], list2Arr(x[1], Dict_wPos)))
    # TF
    tf = T1res.map(lambda x: (x[0], x[1] / x[1].sum()))
    # IDF
    nDoc = corpus.count()
    p = T1res.map(lambda x: (x[0], np.clip(x[1], 0, 1)))
    q = p.map(lambda x: ("nDoc", x[1])) 
    nWinDoc = q.reduceByKey(lambda a, b: a + b)
    idf = np.log(nDoc / nWinDoc.lookup("nDoc")[0])
    # TF-IDF
    tf_idf = tf.map(lambda x: (x[0], x[1] * idf))
    # Normalization
    mean = tf_idf.values().sum() / tf_idf.count()
    std_dev = np.sqrt(tf_idf.map(lambda x: np.square(x[1] - mean)).reduce(lambda a, b: a+b) / float(tf_idf.count()))
    return tf_idf, topWords, mean, std_dev

def normalize(mean, std_dev, tf_idf):
    tf_idf_norm = tf_idf.map(lambda x: (x[0], np.nan_to_num((x[1] - mean)/std_dev))).cache().sortByKey()
    return tf_idf_norm

In [ ]:
tf_idf_train1, tw_train, mean, std = tf_idf("s3://chrisjermainebucket/comp330_A5/TestingDataOneLinePerDoc.txt")
tf_idf_train1 = normalize(mean, std, tf_idf_train1)
tf_idf_train1.cache()

## sort accoding to count
topWords_cw = sorted(tw_train, key=lambda x: (-x[1], x[0]))

topw = sc.parallelize(range(20000))
w_dict = topw.map(lambda x: (topWords_cw[x][0], x))
w_dict.cache().sortByKey()

#Prints 347
w_dict.lookup("applicant")[0]

#Prints 2
w_dict.lookup("and")[0]

#Prints 504
w_dict.lookup("attack")[0]

#Prints 3014
w_dict.lookup("protein")[0]

#Prints 612
w_dict.lookup("car")[0]

# Task 2

Original: $LLH = \sum_i[(y_i\theta^Tx_i - log(1 + e^{\theta^Tx_i})]$,  where: $y_i \in \{0,1\}, x \in R^{20000}, \theta \in R^{20000} $

After regularization: $LLH = \sum_i[(y_i\theta^Tx_i - log(1 + e^{\theta^Tx_i})] + \color{blue}{\beta||\theta||_2^2}$

Thus:  $\frac{\partial LLH}{\partial \theta} = \sum_i[(y_ix_i - \frac{e^{\theta^Tx_i}}{1 + e^{\theta^Tx_i}}x_i] + 2\beta||\theta||_2$

In [ ]:
# calculate gradient for each row(document)
def grad_doc(x,r,beta):
    if "AU" in str(x[0]):
        y_i = 1
    else:
        y_i = 0
    net_i = r.dot(x[1])
    g = np.vectorize(lambda x: -x*y_i + x*(np.exp(net_i)/(1 + np.exp(net_i))))
    return g(x[1]) + 2*beta*r
 
    
# calculate negative llh for each row(document)    
def nllh(x,r):
    if "AU" in str(x[0]):
        y_i = 1
    else:
        y_i = 0
    net_i = r.dot(x[1])
    return -y_i*net_i+np.log(1+np.exp(net_i))


# gradient descent main function
def gd(init_r, tf_idf, bet = 0.0001):
    num_docs = tf_idf.count()
    r = init_r
    delta = 1
    lr = 1
    loss_now = tf_idf.map(lambda x:nllh(x,r)).reduce(lambda a,b: a+b) + bet*np.linalg.norm(r)
    num_epoch = 0
    while delta>0.0001:
        num_epoch += 1
        grad = tf_idf.map(lambda x:grad_doc(x,r,beta = bet)).reduce(lambda a,b: a+b)/num_docs
        r -= lr*grad
        loss_next = tf_idf.map(lambda x: nllh(x,r)).reduce(lambda a,b: a+b) + bet*np.linalg.norm(r)**2
        delta = abs(loss_next - loss_now)
        print("at epoch:", num_epoch, "the negative log likelihood is:", loss_next)
        if (loss_next > loss_now):
            lr = lr / 2
        else:
            lr = lr*1.1
        loss_now = loss_next
    return r

In [ ]:
# using small data set to pretrain model
r_init = np.random.randn(20000)/10
r_pre_trained = gd(r_init, tf_idf_train1)

# using large training set to retrain model
tf_idf_train2, _, _a, _b = tf_idf("s3://chrisjermainebucket/comp330_A5/SmallTrainingDataOneLinePerDoc.txt")
tf_idf_train2 = normalize(mean, std, tf_idf_train2)
tf_idf_train2.cache()
r_trained = gd(r_pre_trained, tf_idf_train2)

top_50 = r_trained.argsort()[-50:][::-1]
w_dict_reverse = w_dict.map(lambda x: (x[1],x[0])).cache().sortByKey()

for idx in top_50:
    print(w_dict_reverse.lookup(idx),"'s parameter is:", r_trained[idx])

# Task 3

In [ ]:
def predict_evaluate(test, r, cut = 0):
    y_true_raw = test.map(lambda x: ([1] if "AU" in str(x[0]) else [0], x[1]))
    y_true_pred = y_true_raw.map(lambda x: (x[0], [1] if r.dot(x[1]) > cut else [0]))
    res = np.array(y_true_pred.collect())
    
    TP = 0 # True positive
    TN = 0 # True negative
    FP = 0 # False positive
    FN = 0 # False negative
    
    FP_indx = []
    
    for index in range(res.shape[0]):
        if ((res[index,0] == 1) and (res[index,1] == 1)):
            TP += 1
        elif ((res[index,0] == 0) and (res[index,1] == 0)):
            TN += 1
        elif ((res[index,0] == 0) and (res[index,1] == 1)):
            FP += 1
            FP_indx.append(index)
        elif ((res[index,0] == 1) and (res[index,1] == 0)):           
            FN += 1
    print(TP,FP,TN,FN)        
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    recall = TP / (TP + FN)
    precision = TP/ (TP + FP)
    F1 = 2 * precision * recall / (precision + recall)
    print("test accuracy:",round(accuracy, 5))
    print("recall:", round(recall,5))
    print("precision", round(precision,5))
    print("F1 score", round(F1,5))
    
    if len(FP_indx) > 0:
        print('There are', len(FP_indx), "false positive cases.")
        for idx in FP_indx:
            FP_webpage = test.take(idx)[idx-1]
            print(FP_webpage[0])
    return accuracy, recall, precision, F1

In [ ]:
tf_idf_test, _, _a, _b = tf_idf("s3://chrisjermainebucket/comp330_A5/TestingDataOneLinePerDoc.txt")
tf_idf_test = normalize(mean, std, tf_idf_test)
acc, rcl, prc, f1 = predict_evaluate(tf_idf_test, r_trained, cut = 5)